## 크롤링(웹 크롤링)
크롤링이란
- 인터넷 상의 웹 페이지 데이터를 자동으로 수집하는 과정.
- 웹 크롤링은 일반적으로 웹 스크래핑과 연관되며, 둘은 종종 혼용되지만 조금 다른 개념. 웹 크롤링은 웹 페이지를 탐색하고 데이터를 수집하는 반면, 웹 스크래핑은 그 페이지에서 특정 정보를 추출하는 데 중점을 둔다.
- 크롤링은 스크래핑을 포함할 수 있다. 크롤링 과정에서 각 페이지를 방문할 때, 스크래핑을 통해 필요한 데이터를 추출할 수 있다.

웹 크롤링에 사용되는 도구
- BeautifulSoup: Python 라이브러리로, HTML 및 XML 문서를 구문 분석하고 데이터를 추출하는 데 사용.

- Scrapy: 웹 크롤링을 위한 Python 프레임워크로, 효율적이고 확장성이 높은 크롤러를 쉽게 만들 수 있다.

- Selenium: 웹 브라우저 자동화 도구로, JavaScript가 동적으로 로드되는 페이지를 크롤링할 때 유용.

- Requests: Python의 HTTP 라이브러리로, 웹 페이지 요청을 쉽게 할 수 있다.

웹 크롤링의 기본 과정
- 크롤러 설정: 크롤러는 특정 웹 페이지를 시작점으로 설정. 이를 '시드(seed)'라고 부르며, 크롤러는 이 시드 URL에서 시작해 다른 페이지로 이동.

- 페이지 요청: 크롤러는 HTTP 요청을 보내 웹 페이지를 요청. 이 과정에서 크롤러는 브라우저처럼 행동하여 웹 서버에서 페이지를 가져온다.

- 데이터 추출: 웹 페이지가 응답되면, 크롤러는 페이지의 HTML을 분석하고 필요한 데이터를 추출. 이 과정에는 BeautifulSoup, Selenium 같은 라이브러리가 사용될 수 있다.

- 링크 추출 및 큐잉: 크롤러는 현재 페이지에서 다른 페이지로 연결되는 링크를 추출하고, 이 링크들을 큐(queue)에 추가하여 다음 크롤링 대상으로 삼는다.

- 반복: 이 과정은 정해진 규칙이나 종료 조건이 충족될 때까지 반복. 예를 들어, 특정 수의 페이지를 크롤링하거나, 주어진 도메인 내에서만 크롤링하도록 설정할 수 있다.

웹 크롤링의 주의사항

- 로봇 배제 표준(robots.txt): 많은 웹사이트는 robots.txt 파일을 통해 크롤러가 접근 가능한 부분과 접근을 제한하는 부분을 명시. 크롤러는 이 규칙을 준수해야 한다.

- 저작권 및 법적 이슈: 모든 웹사이트의 콘텐츠는 저작권의 보호를 받는다. 따라서 크롤링을 통해 수집한 데이터를 어떻게 사용할지에 대한 법적 문제를 주의해야 한다.

- 서버 부하: 지나친 크롤링은 웹 서버에 부하를 줄 수 있다. 크롤링 시에는 서버의 부담을 줄이기 위해 요청 간의 딜레이를 설정하는 것이 좋다.

#### 크롤링 사례

In [ ]:

import requests
from bs4 import BeautifulSoup
from collections import deque
import urllib.robotparser
from urllib.parse import urljoin
import time

# robots.txt 파일을 파싱하고 분석하기 위해서는 urllib.robotparser 모듈을 사용
def can_fetch_url(url):
    robot_parser = urllib.robotparser.RobotFileParser()
    robot_parser.set_url('https://example.com/robots.txt')
    robot_parser.read()
    return robot_parser.can_fetch('*', url) # '*'는 모든 사용자 에이전트를 의미. 반환값은 True 또는 False

def get_response_with_retries(url, max_retries=5, backoff_factor=0.3):
    for i in range(max_retries):
        try:
            response = requests.get(url)
            response.raise_for_status()  # HTTP 응답 코드가 200이 아닌 경우 예외 발생
            return response
        except requests.ConnectionError as e: # 네트워크 문제나 서버가 응답하지 않는 경우
            print(f"Connection error: {e}")
        except requests.HTTPError as e: # 클라이언트 오류(4xx) 또는 서버 오류(5xx) 응답 코드
            print(f"HTTP error: {e}")
        except requests.RequestException as e: # 기타 모든 요청 관련 예외(RequestException)를 처리
            print(f"Request failed: {e}")
        time.sleep(backoff_factor * (2 ** i))  # 지수 백오프의 기본 원리는 재시도할 때마다 대기 시간을 두 배로 늘리는 것. 백오프 시간=기본 시간×(2의 n승)
    return None

start_url = 'https://example.com'
# 이미 방문한 URL들을 저장하는 집합
visited = set() # 집합을 사용하여 중복된 URL 방문을 피할 수 있다.
# 방문할 URL들을 저장하는 큐
queue = deque([start_url]) # deque는 양쪽 끝에서 빠르게 추가 및 제거할 수 있는 양방향 큐

max_urls = 10  # 최대 방문할 URL 수
visited_count = 0 # 현재까지 방문한 URL 수

# BFS(Breadth-First Search) 알고리즘을 사용하여 큐를 통해 웹 페이지를 순차적으로 방문. 각 페이지에서 h1 태그를 추출하고, 새로운 링크를 큐에 추가하여 크롤링을 확장
while queue: # 큐에 방문할 URL이 있는 동안 계속 반복
    if visited_count >= max_urls:
        print(f"Reached the limit of {max_urls} URLs. Stopping the crawler.")
        break

    current_url = queue.popleft() # 큐에서 다음 방문할 URL을 꺼내기
    if current_url in visited: # 이미 방문한 URL이면 다음 반복으로 건너뛰기
        continue

    visited.add(current_url) # 현재 URL을 방문한 URL 집합에 추가
    visited_count += 1

    if not can_fetch_url(current_url): # robots.txt 규칙을 확인하여 현재 URL이 접근 가능한지 확인
        print(f"Blocked by robots.txt: {current_url}") # 접근이 차단된 URL이면 메시지를 출력하고 다음 반복으로 건너뛰기
        continue

    response = get_response_with_retries(current_url)
    if response is None: # 응답이 성공적이지 않으면 메시지를 출력하고 다음 반복으로 건너뛰
        print(f"Failed to retrieve {current_url} after retries")
        continue

    soup = BeautifulSoup(response.text, 'html.parser') # 응답 내용을 BeautifulSoup으로 파싱

    titles = soup.find_all('h1') # 페이지 내의 모든 h1 태그를 찾
    for title in titles: # 각 h1 태그의 텍스트를 출력
        print(title.get_text())

    links = soup.find_all('a', href=True) # 페이지 내의 모든 a 태그에서 href 속성이 있는 링크를 찾
    for link in links:
        href = link['href']
        full_url = urljoin(current_url, href)  # 각 링크의 href 속성을 절대 경로로 변환하고, 큐에 추가
        if full_url not in visited and full_url.startswith('http'):
            queue.append(full_url)

#### BeautifulSoup
- BeautifulSoup은 HTML이나 XML 문서를 파싱하고, 파싱한 데이터에서 원하는 요소를 검색하고 추출하는 데 매우 유용한 도구입니다. 
- BeautifulSoup에서 객체를 찾는 주요 방법에는 find, find_all, select_one, select, find_parents, find_parent, find_next_sibling, find_previous_sibling 등이 있습니다.

검색 방식
- find, find_all: 태그 이름과 속성을 사용하여 요소를 검색합니다.
- select_one, select: CSS 선택자를 사용하여 요소를 검색합니다.

반환 결과:
- find: 첫 번째로 일치하는 요소를 반환합니다.
- find_all: 모든 일치하는 요소를 리스트로 반환합니다.
- select_one: 첫 번째로 일치하는 요소를 반환합니다.
- select: 모든 일치하는 요소를 리스트로 반환합니다.

표현력:
- select_one, select: 더 복잡하고 정교한 선택 조건을 지정할 수 있습니다. 예를 들어, CSS 선택자 문법을 사용하여 클래스, ID, 속성 등을 조합한 검색이 가능합니다.
- find, find_all: 단순한 태그 이름과 속성 조건에 기반한 검색이 주로 사용됩니다.



In [14]:
! pip install requests beautifulsoup4 urllib3 -q

html.parser vs. lxml
- 파이썬에서 HTML 및 XML 문서를 파싱(parsing)하는 라이브러리
- html.parser는 HTML 문서를 파싱하는 데에 적합한 파서. 파이썬의 기본 라이브러리로 제공되며 파이썬 내부적으로 구현되어 있으며, 외부 종속성이 없으므로 파이썬과 함께 설치되는 패키지만 사용할 수 있습니다.
- lxml은 C 언어로 작성된 파이썬 외부 라이브러리로서 HTML 및 XML 문서를 파싱하는 데에 적합하며, 파서 성능이 매우 우수합니다.
- HTML 문서를 파싱하는 경우에는 html.parser를 사용하는 것이 간단하고 편리하며, 대부분의 경우에는 충분한 성능을 제공합니다. 그러나 대용량의 XML 문서나 매우 복잡한 HTML 문서를 파싱해야 하는 경우에는 lxml을 사용하는 것이 더 효율적입니다.

In [29]:
from bs4 import BeautifulSoup

html_content = '<html><body><h1>Title</h1><p class="content">First paragraph.</p><p class="content">Second paragraph.</p></body></html>'
soup = BeautifulSoup(html_content, 'html.parser')
print(soup.prettify())

<html>
 <body>
  <h1>
   Title
  </h1>
  <p class="content">
   First paragraph.
  </p>
  <p class="content">
   Second paragraph.
  </p>
 </body>
</html>



In [30]:
# 첫 번째로 매칭되는 'p' 태그 찾기
first_p = soup.find('p')
print(first_p.get_text())  


First paragraph.


In [25]:
# 모든 'p' 태그 찾기
all_p = soup.find_all('p')
for p in all_p:
    print(p.get_text())

First paragraph.
Second paragraph.


In [26]:
# 클래스가 'content'인 첫 번째 'p' 태그 찾기
first_content_p = soup.select_one('p.content')
print(first_content_p.get_text())

First paragraph.


In [27]:
# 클래스가 'content'인 모든 'p' 태그 찾기
all_content_p = soup.select('p.content')
for p in all_content_p:
    print(p.get_text())

First paragraph.
Second paragraph.


In [38]:
specific_p = soup.find('p', class_='content')
specific_p

<p class="content">First paragraph.</p>

In [33]:
# 특정 'p' 태그의 모든 부모 태그 찾기
specific_p = soup.find('p', class_='content')
parents = specific_p.find_parents()
for parent in parents:
    print(parent.name)

body
html
[document]


In [32]:
# 특정 'p' 태그의 첫 번째 부모 태그 찾기
parent = specific_p.find_parent()
print(parent.name) 

body


In [34]:
# 특정 'p' 태그의 다음 형제 태그 찾기
next_sibling = specific_p.find_next_sibling()
print(next_sibling.get_text())

Second paragraph.


In [35]:
# 특정 'p' 태그의 이전 형제 태그 찾기
previous_sibling = specific_p.find_previous_sibling()
print(previous_sibling)

<h1>Title</h1>


In [36]:
# 특정 'p' 태그 다음에 위치한 모든 태그나 문자열 찾기
next_elements = specific_p.find_next()
print(next_elements)

<p class="content">Second paragraph.</p>


In [37]:
# 특정 'p' 태그 이전에 위치한 모든 태그나 문자열 찾기
previous_elements = specific_p.find_previous()
print(previous_elements) 

<h1>Title</h1>


In [15]:
import requests
from bs4 import BeautifulSoup

# 예시 네이버 뉴스 페이지 URL
url = 'https://news.naver.com'

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

response = requests.get(url, headers=headers)
response

<Response [200]>

In [16]:
response.content

b'\n<!doctype html>\n<html lang="ko">\n\t<head>\n\t\t<title id="browserTitleArea">\xeb\x84\xa4\xec\x9d\xb4\xeb\xb2\x84 \xeb\x89\xb4\xec\x8a\xa4</title>\n\t\t\n\n\n<script>\n\tfunction isMobileDevice() {\n\t\treturn /^.*(iPhone|iPod|iPad|Android).*/.test(navigator.userAgent);\n\t}\n</script>\n<script>\n\t(function () {\n\t\ttry {\n\t\t\tif (isMobileDevice() && isAbleApplyPrefersColorScheme()) {\n\t\t\t\t\n\t\t\t\tdocument.querySelector("html").classList.add("DARK_THEME");\n\t\t\t}\n\t\t} catch(e) {}\n\n\t\tfunction isAbleApplyPrefersColorScheme() {\n\t\t\t\n\t\t\tif (window.matchMedia("(prefers-color-scheme)").matches === false) {\n\t\t\t\treturn false;\n\t\t\t}\n\n\t\t\tvar userAgent = navigator.userAgent;\n\n\t\t\tif (userAgent.indexOf("NAVER") > -1) {\n\t\t\t\t\n\t\t\t\tif (/.*NAVER\\([a-zA-Z]*;\\s[a-zA-Z]*;\\s([0-9]*);/.test(userAgent)) {\n\t\t\t\t\treturn Number(RegExp.$1) >= 1000;\n\t\t\t\t}\n\t\t\t} else {\n\t\t\t\t\n\t\t\t\treturn document.cookie.indexOf("NSCS=1") > -1;\n\t\t\t}

In [ ]:
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')
soup

In [17]:
# 'title' 태그만으로 요소 찾기
news_titles = soup.find('title')
news_titles

<title id="browserTitleArea">네이버 뉴스</title>

In [20]:
# title 태그와 id 속성을 이용하여 원하는 요소를 찾는 방법
title_element = soup.find('title', id='browserTitleArea')
title_element

<title id="browserTitleArea">네이버 뉴스</title>

In [22]:
cats = soup.find_all(class_='Nitem_link_menu')
for idx, cat in enumerate(cats):
    print(f"{idx + 1}: {cat.get_text().strip()}")

1: 언론사별
2: 정치
3: 경제
4: 사회
5: 생활/문화
6: IT/과학
7: 세계
8: 랭킹
9: 신문보기
10: 오피니언
11: TV
12: 팩트체크
13: 알고리즘 안내
14: 정정보도 모음


In [19]:
# CSS 선택자를 사용하여 title 태그 중 ID가 browserTitleArea인 요소를 검색
title_element = soup.select_one('title#browserTitleArea')
title_element

<title id="browserTitleArea">네이버 뉴스</title>

In [23]:
cats = soup.select('.Nitem_link_menu')
for idx, cat in enumerate(cats):
    print(f"{idx + 1}: {cat.get_text().strip()}")

1: 언론사별
2: 정치
3: 경제
4: 사회
5: 생활/문화
6: IT/과학
7: 세계
8: 랭킹
9: 신문보기
10: 오피니언
11: TV
12: 팩트체크
13: 알고리즘 안내
14: 정정보도 모음


string vs. get_text() vs. text
- BeautifulSoup 객체에서 text 속성, get_text() 메서드, string 속성은 모두 HTML 또는 XML 문서에서 텍스트 데이터를 추출하는 데 사용
- text 속성은 해당 태그에서 모든 텍스트 데이터를 가져오며,
- get_text() 메서드도 동일한 결과를 반환합니다.
- string 속성은 해당 태그에서 첫 번째로 발견된 문자열 데이터만 가져옵니다.

| Method      | 특징                                                                               | 사용 사례                                      |
|-------------|----------------------------------------------------------------------------------|-----------------------------------------------|
| `string`    | 자식 텍스트 노드가 하나인 경우에만 해당 텍스트 반환, 여러 노드가 있으면 `None` 반환                | 단일 텍스트 노드만 있는 요소의 텍스트를 추출할 때      |
| `get_text()`| 요소 및 모든 하위 요소의 텍스트를 모두 포함하여 문자열로 반환, `separator` 및 `strip` 옵션 사용 가능 | 요소 내 모든 텍스트를 추출하고, 구분자 및 공백 처리가 필요한 경우 |
| `text`      | 요소 및 모든 하위 요소의 텍스트를 모두 포함하여 문자열로 반환, 추가 매개변수 없음                  | 요소 내 모든 텍스트를 단순하게 추출할 때                   |

string
- 정의: string 속성은 BeautifulSoup 객체의 자식 노드 중 하나의 텍스트 노드만 반환. 해당 요소의 직접적인 텍스트 콘텐츠가 하나일 때만 유용.
- 특징:
요소가 하나의 자식 텍스트 노드를 가지고 있는 경우에만 그 텍스트를 반환.
여러 자식 요소가 있거나 텍스트 노드가 여러 개 있는 경우 None을 반환.

In [44]:
from bs4 import BeautifulSoup

html = """
<div class="example">
    Single text node
</div>
<div class="example">
    <p>Multiple</p> text <span>nodes</span>
</div>
"""

soup = BeautifulSoup(html, 'html.parser')
single_text_node = soup.find('div', class_='example').string
multiple = soup.find_all('div', class_='example')

multiple_text_nodes = soup.find_all('div', class_='example')[1].string

print(single_text_node)  # Output: Single text node
print(multiple,'\n')
print(multiple_text_nodes)  # Output: None


    Single text node

[<div class="example">
    Single text node
</div>, <div class="example">
<p>Multiple</p> text <span>nodes</span>
</div>] 

None


get_text()
- 정의: get_text() 메서드는 요소 및 모든 하위 요소의 텍스트를 모두 추출하여 하나의 문자열로 반환.
- 특징:
요소 내의 모든 텍스트를 포함하여 반환.
기본적으로 하위 요소 사이에 공백을 추가하지만, separator 매개변수를 사용하여 구분자를 지정할 수 있다.
strip=True 옵션을 사용하여 앞뒤 공백을 제거할 수 있다.

In [46]:
from bs4 import BeautifulSoup

html = """
<div class="example">
    Single text node
</div>
<div class="example">
    <p>Multiple</p> text <span>nodes</span>
</div>
"""

soup = BeautifulSoup(html, 'html.parser')

# 기본적으로 `get_text()` 메서드는 공백을 구분자로 사용합니다.
# strip=True 옵션을 사용하여 앞뒤 공백을 제거하고 텍스트를 추출
single_text_node = soup.find('div', class_='example').get_text(strip=True)
multiple_text_nodes = soup.find_all('div', class_='example')[1].get_text(strip=True)

print("Default separator:")
print(single_text_node)  # Output: Single text node
print(multiple_text_nodes)  # Output: Multiple text nodes



Default separator:
Single text node
Multipletextnodes


In [47]:
# `separator` 매개변수를 사용하여 구분자를 지정합니다.
single_text_node_with_separator = soup.find('div', class_='example').get_text(strip=True)
multiple_text_nodes_with_separator = soup.find_all('div', class_='example')[1].get_text(separator=' | ', strip=True)

print("\nCustom separator (' | '):")
print(single_text_node_with_separator)  # Output: Single text node
print(multiple_text_nodes_with_separator)  # Output: Multiple | text | nodes


Custom separator (' | '):
Single text node
Multiple | text | nodes


text
- 정의: text 속성은 요소 및 모든 하위 요소의 텍스트를 모두 포함하는 문자열을 반환. get_text() 메서드와 동일한 기능을 한다.
- 특징:
get_text()와 거의 동일한 결과를 제공.
get_text()와 다르게 추가 매개변수(separator, strip)를 사용할 수 없다.

In [48]:
from bs4 import BeautifulSoup

html = """
<div class="example">
    Single text node
</div>
<div class="example">
    <p>Multiple</p> text <span>nodes</span>
</div>
"""

soup = BeautifulSoup(html, 'html.parser')
single_text_node = soup.find('div', class_='example').text
multiple_text_nodes = soup.find_all('div', class_='example')[1].text

print(single_text_node)  # Output: Single text node
print(multiple_text_nodes)  # Output: Multiple text nodes


    Single text node


Multiple text nodes



정규표현식을 이용한 크롤링 방법 

HTML 가져오기:
- requests 라이브러리를 사용하여 웹 페이지의 HTML을 가져옵니다.
HTML 파싱:
- BeautifulSoup을 사용하여 HTML 문서를 파싱합니다.
정규표현식 사용:
- re 모듈을 사용하여 특정 패턴을 가진 텍스트를 추출합니다.

In [60]:
import requests
from bs4 import BeautifulSoup
import re

html = """
<ul>
  <li>Email: example@example.com</li>
  <li>Contact: contact@sample.org</li>
</ul>
"""
# 이메일 주소 추출
email_pattern = r'[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+'
emails = re.findall(email_pattern, html)
print(emails)


['example@example.com', 'contact@sample.org']


In [62]:
html = '''
<html>
<body>
<div>
Hello,world!
</div>
<div>
<p>
Hello,<b>world!</b>
</p>
</div>
</body>
</html>
'''
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')
bs = soup.body.text
print(bs)



Hello,world!



Hello,world!





In [63]:
import re
text = re.findall('[^\s]+',bs)
for t in text:
    print(t)
# print(re.findall('[A-Za-z!]+',bs))

Hello,world!
Hello,world!


In [64]:
li = ''
for t in text:
    li += t
    li += ' '
print(li)

Hello,world! Hello,world! 


In [99]:
from bs4 import BeautifulSoup
import re # 정규 표현식을 사용할 때
html = """
<ul>
  <li><a href="hoge.html">hoge</li>
  <li><a href="https://example.com/fuga">fuga*</li>
  <li><a href="https://example.com/foo">foo*</li>
  <li><a href="http://example.com/aaa">aaa</li>
</ul>
"""
soup = BeautifulSoup(html, "html.parser")
# 정규 표현식으로 href에서 https인 것 추출하기
li = soup.find_all(href=re.compile(r"^https://"))
for e in li: print(e.attrs['href'])

https://example.com/fuga
https://example.com/foo


#### urllib + bs

In [77]:
import urllib.request as rq

url = 'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=100'

html = rq.urlopen(url)
bs = BeautifulSoup(html, 'html.parser')
print(bs)

<!DOCTYPE html>

<html data-useragent="Python-urllib/3.10" lang="ko">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta contents="always" name="referrer"/>
<meta content="600" http-equiv="refresh"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no" name="viewport">
<meta content="정치 : 네이버 뉴스" property="og:title"/>
<meta content="website" property="og:type"/>
<meta content="https://news.naver.com/section/100" property="og:url"/>
<meta content="https://ssl.pstatic.net/static.news/image/news/ogtag/navernews_800x420_20221201.png" property="og:image"/>
<meta content="국회, 행정, 국방, 외교 등 정치 분야 뉴스 제공" property="og:description"/>
<meta content="네이버" property="og:article:author"/>
<meta content="summary" name="twitter:card"/>
<meta content="정치 : 네이버 뉴스" name="twitter:title"/>
<meta content="네이버 뉴스" name="twitter:site"/>
<meta content="네이버 뉴스" name="twitter:creator"/>
<meta content="https://ssl.pst

In [80]:
text1 = bs.find('p')
print(text1)

<p class="sa_head_layer_p">각 헤드라인의 기사와 배열 순서는 개인화를 반영해 자동 추천되어 제공됩니다. 기사 수량이 표기된 기사 우측 하단의 파란색 아이콘을 클릭하면 기사묶음을 확인할 수 있고 기사묶음과 기사묶음 타이틀도 기사 내용을 기반으로 <strong>자동 추출</strong>됩니다.</p>


In [84]:
text10 = bs.find_all('p')
for t in text10:
    print(t.text)

각 헤드라인의 기사와 배열 순서는 개인화를 반영해 자동 추천되어 제공됩니다. 기사 수량이 표기된 기사 우측 하단의 파란색 아이콘을 클릭하면 기사묶음을 확인할 수 있고 기사묶음과 기사묶음 타이틀도 기사 내용을 기반으로 자동 추출됩니다.
AiRS 추천으로 구성된 뉴스를 제공합니다.
각 언론사의 가장 많이 본 기사 1건을 제공합니다.
“손흥민 다리 부러뜨려야…” 中 내 ‘혐한 정서’ 도 넘었다
바닥에 버린 술 치웠더니… “구청 직원인데, 장사 망하게 해줄게” [영상]
BTS 진 ‘허그 행사’서 기습 뽀뽀… “성추행” 아미 화났다
“가마솥에 들어간 개, 살아남은 개가 보게하다니”...키우던 개 보신탕 끓인 60대
배 아파 병원 갔는데 "변비니 집에 가라"..다음날 사망한 소녀 [헬스톡]
"몸 안 좋아 보신탕 먹으려고"…키우던 개 도살한 60대 입건
"못하겠어요" 70대 호소에도…점심 준비시킨 공무원들
日 후쿠시마원전서 방사선 계측 작업하던 50대 사망…"사인 비공개"
'훈련병 얼차려 사망 사건' 중대장·부중대장 소환 조사
기상학 교수 "올여름 한국 40도 넘는 폭염…8월 이후 폭우 더 걱정"
"우리가 욕받이냐" 의사 돕던 간호사도 '보이콧'…'그들만의 리그' 된 의사 총파업
"이러다 일 내겠네"…'207% 폭등' 주가 무섭게 치솟은 회사
"불길한 징조?"…경포 백사장 늘어선 '죽은 멸치떼' 알고보니
거지 수입이 한달 375만원…정부 "절대 돈주지 마세요"
"얼굴 노출" 결심한 대대장 "죽는 날까지‥" 직진 예고
[단독]"2915원에 거래정지된 주식, 700원에 사실 분 찾습니다"
[삶] "생활비 모자라 강남 집 팔자 했더니 아내가 결사반대한다네요"
"로켓배송 없어질 수 있다" 쿠팡 주장에 공정위 "검색순위 조작 말라는 뜻"
지하철서 '꿀잠' 이준석 포착… "쇼라도 좋으니 좀 따라 해라"
콩나물 줄기에 LED 탑재…갤럭시버즈, 5년 만에 바뀐다
33살 연하와 사랑…70살 할머니 홍학이 알을 낳았다
중국인 때문에 또 비행기 지연…이번엔 30대女 몰래 반입한 '이것' 때문

In [86]:
bs.find('p',class_="sa_head_layer_p")

<p class="sa_head_layer_p">각 헤드라인의 기사와 배열 순서는 개인화를 반영해 자동 추천되어 제공됩니다. 기사 수량이 표기된 기사 우측 하단의 파란색 아이콘을 클릭하면 기사묶음을 확인할 수 있고 기사묶음과 기사묶음 타이틀도 기사 내용을 기반으로 <strong>자동 추출</strong>됩니다.</p>

In [85]:
text2 = bs.find('p').string
print(text2)

None


In [88]:
print(bs.find('title'))

<title>정치 : 네이버 뉴스</title>


In [87]:
print(bs.find('title').string)

정치 : 네이버 뉴스


In [83]:
text3 = bs.find('p').get_text()
print(text3)

각 헤드라인의 기사와 배열 순서는 개인화를 반영해 자동 추천되어 제공됩니다. 기사 수량이 표기된 기사 우측 하단의 파란색 아이콘을 클릭하면 기사묶음을 확인할 수 있고 기사묶음과 기사묶음 타이틀도 기사 내용을 기반으로 자동 추출됩니다.


#### requests + bs

인코딩 에러를 해결

 chardet.detect(response.content)['encoding']은 response.content의 인코딩 방식을 자동으로 감지하여 반환하며
 이 값을 encoding 변수에 저장한 후, response.content를 이 encoding 방식으로 디코딩하여 html 변수에 저장하고 출력

In [91]:
! pip install chardet

   ---------------------------------------- 0.0/199.4 kB ? eta -:--:--
   -------- ------------------------------ 41.0/199.4 kB 991.0 kB/s eta 0:00:01
   ---------------------------------------- 199.4/199.4 kB 3.1 MB/s eta 0:00:00


In [92]:
import requests
import chardet

# HTTP 요청에서 사용될 헤더 정보를 설정합니다.
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36"}

# 웹사이트에서 스크래핑할 URL을 지정합니다.
url = 'https://news.naver.com/main/main.nhn?mode=LSD&mid=shm&sid1=100'

# 지정된 url에 HTTP GET 요청을 전달
response = requests.get(url, headers=headers)

# HTTP 요청이 성공적으로 전달되었다면, 웹사이트의 HTML 코드를 출력합니다.
if response.status_code == 200:
    encoding = chardet.detect(response.content)['encoding']
    # print(response.content.decode(encoding)) # 웹사이트의 HTML 코드를 포함하는 이진(binary) 데이터를 반환
    print(response.text) # 웹사이트의 HTML 코드를 문자열 형태로 반환
else:
    print("HTTP request failed.")

<!doctype html>
<html lang="ko" data-useragent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36">
	<head>
		<meta charset="utf-8">
		<meta http-equiv="X-UA-Compatible" content="IE=edge">
		<meta name="referrer" contents="always">
		<meta http-equiv="refresh" content="600">
		<meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no" />
		<meta property="og:title" content="정치 : 네이버 뉴스">
		<meta property="og:type" content="website">
		<meta property="og:url" content="https://news.naver.com/section/100">
		<meta property="og:image" content="https://ssl.pstatic.net/static.news/image/news/ogtag/navernews_800x420_20221201.png">
		<meta property="og:description" content="국회, 행정, 국방, 외교 등 정치 분야 뉴스 제공">
		<meta property="og:article:author" content="네이버">
		<meta name="twitter:card" content="summary">
		<meta name="twitter:title" content="정치 : 네이버 뉴스">
		<meta nam

In [93]:
print(response.text)

<!doctype html>
<html lang="ko" data-useragent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36">
	<head>
		<meta charset="utf-8">
		<meta http-equiv="X-UA-Compatible" content="IE=edge">
		<meta name="referrer" contents="always">
		<meta http-equiv="refresh" content="600">
		<meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no" />
		<meta property="og:title" content="정치 : 네이버 뉴스">
		<meta property="og:type" content="website">
		<meta property="og:url" content="https://news.naver.com/section/100">
		<meta property="og:image" content="https://ssl.pstatic.net/static.news/image/news/ogtag/navernews_800x420_20221201.png">
		<meta property="og:description" content="국회, 행정, 국방, 외교 등 정치 분야 뉴스 제공">
		<meta property="og:article:author" content="네이버">
		<meta name="twitter:card" content="summary">
		<meta name="twitter:title" content="정치 : 네이버 뉴스">
		<meta nam

In [95]:
# # 한글만 출력
import re
texts = response.text
result = re.findall('[가-힣]+',texts)
print(' '.join(result))

정치 네이버 뉴스 국회 행정 국방 외교 등 정치 분야 뉴스 제공 네이버 정치 네이버 뉴스 네이버 뉴스 네이버 뉴스 국회 행정 국방 외교 등 정치 분야 뉴스 제공 정치 네이버 뉴스 로그 전송 최대치 도달 본문 바로가기 뉴스 연예 스포츠 날씨 프리미엄 검색 언론사별 정치 경제 사회 생활 문화 과학 세계 랭킹 신문보기 오피니언 팩트체크 알고리즘 안내 정정보도 모음 금 전체 언론사 뉴스스탠드 라이브러리 정치 대통령실 국회 정당 북한 행정 국방 외교 정치일반 뉴스 알고리즘 프리미엄콘텐츠 헤드라인 뉴스 안내 각 헤드라인의 기사와 배열 순서는 개인화를 반영해 자동 추천되어 제공됩니다 기사 수량이 표기된 기사 우측 하단의 파란색 아이콘을 클릭하면 기사묶음을 확인할 수 있고 기사묶음과 기사묶음 타이틀도 기사 내용을 기반으로 자동 추출 됩니다 알고리즘 자세히 보기 닫기 안철수 국힘 전대룰 민심 반영에 민심 받든다는 말 민망 안철수 국힘 전대룰 민심 반영에 민심 받든다는 말 민망 국민의힘 차기 당권 주자로 꼽히는 안철수 의원이 일 이번 전당대회에 적용할 경선 규칙을 당원 투표 와 일반 국민 여론조사 로 변경하기로 한 데 대해 라는 비율은 민심을 받든다는 말을 하기조차 민 뉴스 개의 관련뉴스 더보기 이재명 대북송금 희대의 조작 사건으로 밝혀질 것 이재명 대북송금 희대의 조작 사건으로 밝혀질 것 이재명 더불어민주당 대표가 쌍방울그룹 대북송금 관련 제 자 뇌물수수 혐의에 대해 희대의 조작 사건으로 결국 밝혀질 것 이라고 말했습니다 이 대표는 오늘 일 서울중앙지방법원에서 열리는 공직선거법 관련 재판 출 개의 관련뉴스 더보기 한 총리 의사 집단 휴진 예고에 환자 곁에 머물러 달라 한 총리 의사 집단 휴진 예고에 환자 곁에 머물러 달라 한덕수 국무총리는 일 일부 의대 교수과 개원의가 집단 휴진을 예고한 것과 관련해 지금의 결정을 거두고 환자곁에 머물러 주기를 간곡히 부탁드린다 고 밝혔다 한 총리는 이날 서울 신대방동 서울보라매병원에서 의사집 파이낸셜뉴스 개의 관련뉴스 더보기 추경호 경찰청장

In [96]:
import requests
from bs4 import BeautifulSoup

url = "https://serieson.naver.com/v3/movie/ranking/realtime"

# HTTP GET 요청을 보냅니다.
response = requests.get(url)
response

<Response [200]>

In [97]:
import requests
from bs4 import BeautifulSoup

url = "https://serieson.naver.com/v3/movie/ranking/realtime"

# HTTP GET 요청을 보냅니다.
response = requests.get(url)

# HTTP 요청이 성공적으로 전달되었다면, 웹사이트의 HTML 코드를 가져옵니다.
if response.status_code == 200:
    # HTML 코드를 파싱합니다.
    soup = BeautifulSoup(response.text, "html.parser")
# print(soup)
soup.find("span", class_="Title_title__s9o0D").text

'인사이드 아웃(패키지상품 : 더빙판 + 부가영상 추가증정)'

In [98]:
import requests
from bs4 import BeautifulSoup
req = requests.get("https://serieson.naver.com/v3/movie/ranking/realtime")
html = req.content.decode('utf-8')
soup = BeautifulSoup(html, 'html.parser')
list = soup.find_all("span", class_="Title_title__s9o0D")
for i, l in enumerate(list[:10]):
    print(f'{i+1}: {l.text}')

1: 인사이드 아웃(패키지상품 : 더빙판 + 부가영상 추가증정)
2: 블랙북
3: 악마와의 토크쇼
4: 악마와의 토크쇼
5: 소울메이트
6: 매드맥스: 분노의 도로(2.35:1 극장판)
7: 파묘
8: 원피스 필름 레드
9: 매드맥스: 분노의 도로
10: 한공주


### CSS 선택자
- 원하는 정보만 선별하여 수집하고 싶을 때 css선택자를 활용할 수 있음
- (CSS 선택자 설명 추가)
- F12 >> 수집하고 싶은 부분 클릭 >> 태그 선택 >> copy Selector
- BeautifulSoup의 select_one, select 활용

In [11]:
import requests
from bs4 import BeautifulSoup

url = 'https://naver.com'
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'lxml')

In [18]:
soup.select_one('span.service_name')

In [13]:
# ID가 'main'인 요소 찾기
main_div = soup.select_one('#shortcutArea > ul > li > a > span.service_name')
print(main_div)

# 클래스가 'content'인 첫 번째 <p> 태그 찾기
# first_content_p = soup.select_one('#shortcutArea > ul > li:nth-child(1) > a > span.service_name')
# print(first_content_p.get_text())

None
